In [ ]:
import csv
import sqlite3 as sql
import pandas as pd
import pprint
from datetime import datetime as dtm
import time
import re
import json

In [ ]:
###
## And for newer versions of Python (3.6+, https://www.python.org/dev/peps/pep-0498/ purely for completeness), you can use the newer string formatting, ie.

def get_time_formatted():
    is_now = time.strftime('%Y-%m-%d %H:%M:%S')
    # is_now_formatted = str({is_now:%Y-%m-%d %H:%M:%S})
    return is_now

In [ ]:
get_time_formatted()

In [ ]:
### Delete all trailing whitespaces. – Documentation:
#  https://stackoverflow.com/questions/40950310/strip-trim-all-strings-of-a-dataframe
def trim_all_columns(df):
    """
    Trim whitespace from ends of each value across all series in dataframe
    """
    trim_strings = lambda x: x.strip() if isinstance(x, str) else x
    return df.applymap(trim_strings)

## Préparation 

In [ ]:
### Ajouter le dossier parent dans le chemin de recherche des modules 
# Solution retenue depuis cette doc:
# https://stackoverflow.com/questions/714063/importing-modules-from-parent-folder/11158224
## Autres solutions également proposées

import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir) 

In [ ]:
parentdir

In [ ]:
currentdir

In [ ]:
### Enseignants des collèges
file_path = "[FILE PATH]/"
file_name = "pfap"

In [ ]:
file_suffix = '.xls'  # '.xlsx'  '.csv' '.xls'

### Code à exécuter chaque fois à partir d'ici

In [ ]:
file_address = file_path + file_name + file_suffix
file_address

### Créée l'objet *pd_data* et la liste des colonnes: *columns_new_list*

In [ ]:
pcsv = re.compile('.csv$')
pxls = re.compile('.xlsx$|.xls$')

try:
    if pcsv.search(file_address):
        delimiter = ''
        quotechar = ''
        with open(file_address, newline='') as csvfile:
            # Augmented from 1024 to have enough text when many columns
            dialect = csv.Sniffer().sniff(csvfile.read(8192))
            delimiter = dialect.delimiter
            quotechar = dialect.quotechar
        pd_data = pd.read_csv(file_address, delimiter=delimiter, quotechar=quotechar, index_col=False, low_memory=False)
    elif pxls.search(file_address):
        pd_data = pd.read_excel(file_address)
    else:
        print('No data available !')     
        
    # This part adds a column pk_table with value 1 to number of rows
    # It will be used to sort the rows but will not be imported in the GVS data tables
    i = 1
    pk_list = []
    limit = len(pd_data)
    while i <= limit :
        pk_list.append(i)
        i = i + 1
    pd_data.insert(0, 'pk_table', pk_list)
    
    ### builds a list with new column label, Pandas value type and original column label
    columns=pd_data.dtypes.apply(lambda x: x.name).to_dict()
    columns_new_list = []
    i = 0
    for key, value in columns.items():
        temp = [value,key]
        columns_new_list.append(['col_'+ str(i)] + temp)
        i = i + 1
        

except Exception as e:
    # e = sys.exc_info()[0]
    print( e )


In [ ]:
columns_new_list[:40]    # [68:77]

In [ ]:
pd_data.head()

In [ ]:
### Applying this function strips all spaces in columns of type string

pd_data = trim_all_columns(pd_data)


In [ ]:
len(pd_data)

In [ ]:
len(pd_data.columns)

In [ ]:
conn = sql.connect('enseignants.sqlite')

In [ ]:
pd_data.to_sql('pfap', conn, if_exists='replace', index=False, )

In [ ]:
pd.read_sql('select * from pfap', conn)